In [ ]:
#default_exp media

# Convert HTML and Images to MDX
> Make allowances for plots and dataframes in MDX

In [ ]:
#export
from nbprocess.processor import *
from fastcore.imports import Path
from fastcore.basics import nested_idx
from html.parser import HTMLParser

from nbprocess.test_utils import *

In [ ]:
#export
class _HTMLdf(HTMLParser):
    "HTML Parser that finds a dataframe."
    df,scoped = False,False
    def handle_starttag(self, tag, attrs):
        if tag == 'style' and 'scoped' in dict(attrs): self.scoped=True
    def handle_data(self, data):
        if '.dataframe' in data and self.scoped: self.df=True
    def handle_endtag(self, tag):
        if tag == 'style': self.scoped=False
                
    @classmethod
    def search(cls, x):
        parser = cls()
        parser.feed(x)
        return parser.df

@preprocess_cell
def HTMLEscape(cell):
    "Place HTML in a codeblock and surround it with a <HTMLOutputBlock> component."
    if cell.cell_type !='code': return
    for o in cell.outputs:
        if nested_idx(o, 'data', 'text/html'):
            cell.metadata.html_output = True
            html = o['data']['text/html']
            cell.metadata.html_center = not _HTMLdf.search(html)
            o['data']['text/html'] = '```html\n'+html.strip()+'\n```'

By default, HTML is incompatible with MDX.  We place HTML in a code block and wrap it with the a custom component so that the static site generator can render it.

In [ ]:
c, _ = run_preprocessor([HTMLEscape], '../tests/docs_test.ipynb')
assert '<HTMLOutputBlock' in c and '</HTMLOutputBlock>' in c and 'center' not in c
assert '```html\n<div>' in c and '</div>\n```' in c

In [ ]:
#export
class ImageSave(Preprocessor):
    "Saves images stored as bytes in notebooks to disk."
    def preprocess(self, nb, resources):
        meta = resources.get('metadata', {})
        nb_name,nb_path = meta.get('name'),meta.get('path')
        outfiles = resources.get('outputs')
        if nb_name and outfiles:
            resources['fmap'] = {}
            for k,v in outfiles.items():
                dest = Path(nb_path)/f'_{nb_name}_files/{k}'
                dest.parent.mkdir(exist_ok=True)
                dest.write_bytes(v)
                resources['fmap'][f'{k}'] = f'_{nb_name}_files/{k}'       
        return nb, resources

class ImagePath(Preprocessor):
    "Changes the image path to the location where `ImageSave` saved the files."
    def preprocess_cell(self, cell, resources, index): 
        fmap = resources.get('fmap')
        if fmap:
            for o in cell.get('outputs', []):
                fnames = o.get('metadata', {}).get('filenames', {})
                for k,v in fnames.items():
                    fnames[k] = fmap.get(v,v)
        return cell, resources

`ImageSave` and `ImagePath` must be used together to extract and save images from notebooks and change the path.  This is necessary to enable compatiblity with certain types of plotting libraries like matplotlib.

In [ ]:
c, _ = run_preprocessor([ImageSave, ImagePath], '../tests/docs_test.ipynb')
assert '![png](_docs_test_files/output_6_0.png)' in c